<a href="https://colab.research.google.com/github/NapsterZ4/saturdays_second_edition/blob/master/cataratas_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Montaje desde google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importando librerías

In [16]:
import tensorflow as tf
import os
import numpy as np

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Cargando datasets

## Etiquetado y extrayendo información

In [17]:
files = []
labels = []

for dirname, _, filenames in os.walk('/content/drive/My Drive/Colab Notebooks/cataratas_images'):
  for filename in filenames:
    if filename.endswith('.png'):
      files.append(os.path.join(dirname, filename))
      l = np.zeros(4)

      if filename.startswith('NL'):
        l[0] = 1
      
      elif filename.startswith('ca'):
        l[1] = 1

      elif filename.startswith('Gl'):
        l[2] = 1

      elif filename.startswith('Re'):
        l[3] = 1
      
      labels.append(1)

print(len(labels), len(files))

601 601


## Train Test